# OpenAI Function Calling In LangChain

In [40]:
import os
import openai
from helper import show_response

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [4]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [5]:
foo.name

'Joe'

In [6]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [7]:
foo.age

'bar'

In [8]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [9]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [10]:
foo_p.name

'Jane'

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [11]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing

In [12]:
class Class(BaseModel):
    students: List[pUser]

In [13]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [14]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


In [15]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [18]:
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

In [19]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [20]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [21]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next cell is expected to generate an error.</p>

In [22]:
convert_pydantic_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [23]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [24]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [27]:
from langchain_openai import ChatOpenAI

In [28]:
model = ChatOpenAI()

In [29]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 70, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BxI09Ii78BnV6pdWPgBJXte64Jl2x', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--4660b025-e9b7-4a56-8bce-24f2b215ecca-0', usage_metadata={'input_tokens': 70, 'output_tokens': 17, 'total_tokens': 87, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [30]:
model_with_function = model.bind(functions=[weather_function])

In [31]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 69, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BxI0TTJptDvKtxvcWC5RoG0H4nRVb', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--0604bf30-9db8-4d2f-9671-7bb05ef91ac3-0', usage_metadata={'input_tokens': 69, 'output_tokens': 17, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Forcing it to use a function

We can force the model to use a function

In [32]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [33]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 79, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BxI0Y23PtbLMXAeg2rSzcQGe4YTLs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--dc805223-f66d-4fc6-a3ca-d110dc77c7e1-0', usage_metadata={'input_tokens': 79, 'output_tokens': 7, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [34]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 74, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BxI0amFmNiKhhZF8qa1YWBuGcFo96', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7dcf7535-5444-46eb-9d6d-4bca122c4859-0', usage_metadata={'input_tokens': 74, 'output_tokens': 7, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [35]:
from langchain.prompts import ChatPromptTemplate

In [36]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [37]:
chain = prompt | model_with_function

In [41]:
show_response(chain.invoke({"input": "what is the weather in sf?"}))

{
  "content": "",
  "additional_kwargs": {
    "function_call": {
      "arguments": "{\"airport_code\":\"SFO\"}",
      "name": "WeatherSearch"
    },
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 17,
      "prompt_tokens": 75,
      "total_tokens": 92,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_name": "gpt-3.5-turbo-0125",
    "system_fingerprint": null,
    "id": "chatcmpl-BxI1KFOKIMDXowHpq3soDOeGPbRmn",
    "service_tier": "default",
    "finish_reason": "function_call",
    "logprobs": null
  },
  "type": "ai",
  "name": null,
  "id": "run--eb949fd6-d930-47e4-9841-c9bbe2aa8702-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "inpu

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [42]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [43]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [44]:
model_with_functions = model.bind(functions=functions)

In [45]:
show_response(model_with_functions.invoke("what is the weather in sf?"))

{
  "content": "",
  "additional_kwargs": {
    "function_call": {
      "arguments": "{\"airport_code\":\"SFO\"}",
      "name": "WeatherSearch"
    },
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 17,
      "prompt_tokens": 116,
      "total_tokens": 133,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_name": "gpt-3.5-turbo-0125",
    "system_fingerprint": null,
    "id": "chatcmpl-BxI1dDTFTKWjLrjZSWgefMyafIvsM",
    "service_tier": "default",
    "finish_reason": "function_call",
    "logprobs": null
  },
  "type": "ai",
  "name": null,
  "id": "run--990a747b-9b6c-44ac-b879-f25479dc9cb0-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "in

In [46]:
show_response(model_with_functions.invoke("what are three songs by taylor swift?"))

{
  "content": "",
  "additional_kwargs": {
    "function_call": {
      "arguments": "{\"artist_name\":\"taylor swift\",\"n\":3}",
      "name": "ArtistSearch"
    },
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 22,
      "prompt_tokens": 118,
      "total_tokens": 140,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_name": "gpt-3.5-turbo-0125",
    "system_fingerprint": null,
    "id": "chatcmpl-BxI1lZcusXBjcLu4eW9M8cBHO7yfj",
    "service_tier": "default",
    "finish_reason": "function_call",
    "logprobs": null
  },
  "type": "ai",
  "name": null,
  "id": "run--d8a63913-84c8-4edd-9c44-247f8c9bfee4-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metad

In [48]:
show_response(model_with_functions.invoke("hi!"))

{
  "content": "Hello! How can I assist you today?",
  "additional_kwargs": {
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 10,
      "prompt_tokens": 111,
      "total_tokens": 121,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_name": "gpt-3.5-turbo-0125",
    "system_fingerprint": null,
    "id": "chatcmpl-BxI21g8ivFZQH9jr0HqYYkETC8x1a",
    "service_tier": "default",
    "finish_reason": "stop",
    "logprobs": null
  },
  "type": "ai",
  "name": null,
  "id": "run--deef3e6c-50da-4b00-a118-f6245569843c-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 111,
    "output_tokens": 10,
    "total_tokens": 121,
    "input_token